In [1]:
from util import *

import numpy as np
from scipy.interpolate import PchipInterpolator
import matplotlib.pyplot as plt

In [2]:
A = []
ns = list(range(2, 12, 2)) + list(range(16, 30, 3))

for n in ns:
    print(f'n_piece: {n}')
    ori, d4t = evalate(cgb, 'mushrooms', n_piece=n, only_enc=True)
    A.append(d4t)

print(ns)
print(A)

n_piece: 2
ENC: 99.938
n_piece: 4
ENC: 100.000
n_piece: 6
ENC: 100.000
n_piece: 8
ENC: 99.938
n_piece: 10
ENC: 100.000
n_piece: 16
ENC: 100.000
n_piece: 19
ENC: 100.000
n_piece: 22
ENC: 100.000
n_piece: 25
ENC: 100.000
n_piece: 28
ENC: 100.000
[2, 4, 6, 8, 10, 16, 19, 22, 25, 28]
[0.9993846153846154, 1.0, 1.0, 0.9993846153846154, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [3]:
evalate_all(RandomForestClassifier)

iris
Vanilla: 100.000
ENC: 100.000

mushrooms
Vanilla: 100.000
ENC: 100.000

cod-rna
Vanilla: 95.868
ENC: 95.582

covtype
Vanilla: 95.573
ENC: 95.445

sensorless
Vanilla: 99.838
ENC: 99.863

heart
Vanilla: 87.037
ENC: 83.333

dna
Vanilla: 94.266
ENC: 94.519

madelon
Vanilla: 70.667
ENC: 69.000



In [7]:
evalate_all(DecisionTreeClassifier, ['mushrooms'])

mushrooms
Vanilla: (depth: 7) 100.000
ENC: (depth: 7) 100.000



In [8]:
(6+6+7+7+7)/5

6.6